# Wrfcf Average Tmax

In [1]:
# imports
import numpy
import Nio
import Ngl
from IPython import display
import netCDF4 as nc
from datetime import date
from datetime import timedelta
import xarray as xr
import xesmf as xe

In [2]:
# reading in wrfcf tmaxes, change to tmins to see tmins
wrf_05 = xr.open_dataset('./wrf_05_cpc-Copy1.nc')
wrf_06 = xr.open_dataset('./wrf_06_cpc-Copy1.nc')
wrf_07 = xr.open_dataset('./wrf_07_cpc-Copy1.nc')
wrf_08 = xr.open_dataset('./wrf_08_cpc-Copy1.nc')
wrf_09 = xr.open_dataset('./wrf_09_cpc-Copy1.nc')
wrf_10 = xr.open_dataset('./wrf_10_cpc-Copy1.nc')
wrf_11 = xr.open_dataset('./wrf_11_cpc-Copy1.nc')
wrf_12 = xr.open_dataset('./wrf_12_cpc-Copy1.nc')
wrf_13 = xr.open_dataset('./wrf_13_cpc-Copy1.nc')
wrf_14 = xr.open_dataset('./wrf_14_cpc-Copy1.nc')
wrf_15 = xr.open_dataset('./wrf_15_cpc-Copy1.nc')
#wrf_16 = xr.open_dataset('./wrf_16.nc')
#wrf_17 = xr.open_dataset('./wrf_17.nc')
#wrf_17['maxes'] = wrf_17['T_2m']

#wrf_lst = [wrf_05, wrf_06, wrf_07, wrf_08, wrf_09, wrf_10, wrf_11, wrf_12, wrf_13, wrf_14, wrf_15, wrf_16, wrf_17]
wrf_lst = [wrf_05, wrf_06, wrf_07, wrf_08, wrf_09, wrf_10, wrf_11, wrf_12, wrf_13, wrf_14, wrf_15]

In [3]:
max(wrf_15['maxes'].data.flatten())

300.6074555188385

In [4]:
# averaging
sum_arr = wrf_05['maxes'].data
for i in wrf_lst[1:12]:
    sum_arr += i['maxes'].data
avg = sum_arr / 11 #13
avg

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [5]:
# for reference
numpy.unique(avg), numpy.unique(sum_arr), numpy.unique(wrf_05['maxes'])

(array([  0.        , 260.82634117, 261.39797193, ..., 300.00575803,
        300.12037843, 300.22384672]),
 array([   0.        , 2869.0897529 , 2875.37769119, ..., 3300.06333829,
        3301.32416268, 3302.46231396]),
 array([   0.        , 2869.0897529 , 2875.37769119, ..., 3300.06333829,
        3301.32416268, 3302.46231396]))

In [6]:
# change 0.0 to NaNs
avg[avg == 0.0] = 'nan'

In [20]:
# janky way to find average using livneh coords, subtracting wrf avg/pt by random 
# livneh to create nans then adding livneh avg/pt back so we can remove wrf pts not in common
fn = '/cw3e/mead/projects/cwp103/scratch/ldehaan/CPC/tmax.2017.nc'
liv = xr.open_dataset(fn)
livran = liv['tmax'][35]
sub = numpy.subtract(avg, livran)
new_add = numpy.add(sub, livran)
numpy.unique(avg), numpy.unique(sub), numpy.unique(new_add)
sum(~numpy.isnan(new_add.data.flatten()))

2041

In [8]:
# convert dif_arr to masked array to overlook NaNs
import numpy.ma as ma
m_out = ma.masked_invalid(new_add)
m_out

masked_array(
  data=[[--, --, --, ..., --, --, --],
        [--, --, --, ..., --, --, --],
        [--, --, --, ..., --, --, --],
        ...,
        [--, --, --, ..., --, --, --],
        [--, --, --, ..., --, --, --],
        [--, --, --, ..., --, --, --]],
  mask=[[ True,  True,  True, ...,  True,  True,  True],
        [ True,  True,  True, ...,  True,  True,  True],
        [ True,  True,  True, ...,  True,  True,  True],
        ...,
        [ True,  True,  True, ...,  True,  True,  True],
        [ True,  True,  True, ...,  True,  True,  True],
        [ True,  True,  True, ...,  True,  True,  True]],
  fill_value=1e+20)

In [9]:
# convert from K to C
m_out = m_out - 273.15
min(numpy.unique(m_out))

-12.323658827608995

In [10]:
# numerical average over entire region
numpy.mean(m_out.flatten())

6.491365927113625

# workstation
wks_type = "png"
wks = Ngl.open_wks(wks_type,"wrfcf17_avg_tmin_NG-GR-NQ-BD")

# defining variables
fn = '/cw3e/mead/projects/cwp103/production/output/2017020500/cf/NG-GR-NQ-BD/wrfcf_d02_2017-02-05_22_00_00.nc'
cdf_file1 = Nio.open_file(fn, "r")
temp = m_out
tlo = cdf_file1.variables["lon"][:].flatten()
tla = cdf_file1.variables["lat"][:].flatten()
tmax_lon  =  wrf_05['lon'].data.flatten()
tmax_lat  =  wrf_05['lat'].data.flatten()
tmax_nlon =  len(tmax_lon)
tmax_nlat =  len(tmax_lat)

# XY of overlay plot
resources = Ngl.Resources()

# fill missing values as needed
if hasattr(temp,"_FillValue"):
    resources.sfMissingValueV = temp[0]._FillValue

resources.sfXCStartV = min(tmax_lon)
resources.sfXCEndV   = max(tmax_lon)
resources.sfYCStartV = min(tmax_lat)
resources.sfYCEndV   = max(tmax_lat)

# cmap
resources.cnLevelSelectionMode = "ManualLevels"
resources.cnMinLevelValF       = -20
resources.cnMaxLevelValF       = 32
resources.cnLevelSpacingF      = 4

# plot styling
resources.mpFillOn              = True
resources.mpFillAreaSpecifiers  = ["Water","Land","USStatesWater"]
resources.mpSpecifiedFillColors = [0,0,0]
resources.mpAreaMaskingOn       = True            
resources.mpMaskAreaSpecifiers  = "USStatesLand"
resources.cnFillDrawOrder       = "Predraw" 
Ngl.set_values(wks,resources)
resources.tiXAxisString = "~F25~Longitude"
resources.tiYAxisString = "~F25~Latitude"
resources.cnFillOn              = True    
resources.cnLineLabelsOn        = False  
resources.cnInfoLabelOn         = False  
resources.nglSpreadColorEnd     = -2
resources.mpProjection = "LambertEqualArea" 
resources.mpCenterLonF = (tmax_lon[tmax_nlon-1] + tmax_lon[0])/2
resources.mpCenterLatF = (tmax_lat[tmax_nlat-1] + tmax_lat[0])/2
resources.mpLimitMode = "LatLon"
resources.mpMinLonF   = min(tlo)
resources.mpMaxLonF   = max(tlo)
resources.mpMinLatF   = min(tla)
resources.mpMaxLatF   = max(tla)
resources.mpPerimOn   = True
resources.cnLinesOn   = False
resources.mpOutlineBoundarySets = "GeophysicalAndUSStates"
resources.pmTickMarkDisplayMode = "Never"
resources.tiMainString = "~F26~Wrfcf Average Tmin, 2017-02-05 to 2017-02-17, NG-GR-NQ-BD"
resources.lbOrientation  = "Horizontal"
resources.lbTitleString  = "~F25~Temp (C)"   

# plot
map = Ngl.contour_map(wks,temp,resources)

# clean up
del map
del resources
Ngl.end()

# display plot (needs further cleaning later)
display.Image("./wrfcf17_avg_tmin_NG-GR-NQ-BD.png")

fin.